In [44]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [45]:
import pandas as pd
import wandb

api = wandb.Api()

entity = "tunnels-ssl"
project = "01.23"
# Get all runs for the specified project
runs = api.runs(f"{entity}/{project}")


summary_list, config_list, name_list = [], [], []
for run in runs:
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files
    summary_list.append(run.summary._json_dict["test/avg_acc_tag"])

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append({k: v for k, v in run.config.items() if not k.startswith("_")})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame(
    {"avg_acc_tag": summary_list, "config": config_list, "name": name_list}
)
config_df = pd.json_normalize(runs_df["config"])
df = pd.concat([runs_df.drop(columns=["config"]), config_df], axis=1)
df["data.dataset"] = df["data.datasets"].apply(lambda x: x[0])


def get_real_name(row):
    if row["data.exemplars.num_exemplars"] > 0:
        return "replay"
    return row["training.approach.name"]


def is_regularized(row):
    if row["training.vcreg.var_weight"] == 0 and row["training.vcreg.cov_weight"] == 0:
        return False
    return True


df["approach"] = df.apply(get_real_name, axis=1)
df["is_regularized"] = df.apply(is_regularized, axis=1)

In [46]:
df = df[df["training.vcreg.var_weight"] > 0]
filtered_df = df[
    [
        "avg_acc_tag",
        "approach",
        # "training.vcreg.var_weight",
        # "training.vcreg.cov_weight",
        "misc.seed",
        "data.num_tasks",
        "model.network",
        "data.dataset",
        "is_regularized",
    ]
]
filtered_df = filtered_df.rename(
    columns={
        # "test_acc": "avg_acc_tag",
        "data.num_tasks": "num_tasks",
        "data.dataset": "dataset",
        "model.network": "network",
        "misc.seed": "seed",
    }
)

In [47]:
filtered_df

,avg_acc_tag,approach,seed,num_tasks,network,dataset,is_regularized
0,21.45,ewc,1,10,convnext_tiny,cifar100_fixed,True
1,29.68,replay,1,10,convnext_tiny,imagenet_subset_kaggle,True
2,14.10,ewc,1,20,convnext_tiny,cifar100_fixed,True
3,22.08,ewc,0,20,convnext_tiny,imagenet_subset_kaggle,True
4,14.60,ewc,0,20,convnext_tiny,cifar100_fixed,True
5,30.91,ewc,1,5,convnext_tiny,cifar100_fixed,True
6,20.53,ewc,0,10,convnext_tiny,cifar100_fixed,True
7,24.99,replay,2,20,convnext_tiny,cifar100_fixed,True
8,33.24,ewc,1,10,convnext_tiny,imagenet_subset_kaggle,True
9,27.64,replay,0,20,convnext_tiny,imagenet_subset_kaggle,True


In [48]:
from pathlib import Path

import pandas as pd
import numpy as np

# for csv in Path(
#     "/Users/piotr.hondra/Documents/mgr/cl-adaptation/notebooks/csvs_new/resnet34_skips"
# ).rglob("*.csv"):
#     df = pd.read_csv(csv)
#     df["model.network"] = csv.parts[-2]
#     df.to_csv("all.csv", mode="a", header=False)

In [49]:
all_df = pd.read_csv("all.csv")

In [50]:
df_noreg = all_df[all_df["is_regularized"] == False]

In [51]:
pivot_table = pd.pivot_table(
    df_noreg,
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    fill_value=0,
)
pivot_table

dataset                   cifar100_fixed                          \
num_tasks                             5           10          20   
approach   is_regularized                                          
ewc        False              26.33±3.26  14.57±2.38   6.89±2.62   
finetuning False              23.38±2.88  13.43±1.90   6.61±1.65   
lwf        False              39.98±0.74  27.64±2.30  20.56±3.11   
replay     False              37.58±3.12  32.00±3.79  28.51±3.28   

dataset                   imagenet_subset_kaggle              
num_tasks                                     10          20  
approach   is_regularized                                     
ewc        False                      17.69±3.99   8.91±1.92  
finetuning False                      14.40±2.89   6.72±0.67  
lwf        False                      39.60±4.20  24.61±5.03  
replay     False                      36.57±2.17  32.39±2.94

In [52]:
filtered_df

,avg_acc_tag,approach,seed,num_tasks,network,dataset,is_regularized
0,21.45,ewc,1,10,convnext_tiny,cifar100_fixed,True
1,29.68,replay,1,10,convnext_tiny,imagenet_subset_kaggle,True
2,14.10,ewc,1,20,convnext_tiny,cifar100_fixed,True
3,22.08,ewc,0,20,convnext_tiny,imagenet_subset_kaggle,True
4,14.60,ewc,0,20,convnext_tiny,cifar100_fixed,True
5,30.91,ewc,1,5,convnext_tiny,cifar100_fixed,True
6,20.53,ewc,0,10,convnext_tiny,cifar100_fixed,True
7,24.99,replay,2,20,convnext_tiny,cifar100_fixed,True
8,33.24,ewc,1,10,convnext_tiny,imagenet_subset_kaggle,True
9,27.64,replay,0,20,convnext_tiny,imagenet_subset_kaggle,True


,num_tasks,approach,is_regularized,avg_acc_tag,dataset,seed,network
0,5,finetuning,False,25.72,cifar100_fixed,1,convnext_tiny
2,5,replay,False,34.70,cifar100_fixed,1,convnext_tiny
4,5,ewc,False,29.55,cifar100_fixed,1,convnext_tiny
6,5,lwf,False,40.63,cifar100_fixed,1,convnext_tiny
8,10,finetuning,False,15.57,cifar100_fixed,1,convnext_tiny
...,...,...,...,...,...,...,...
6,10,lwf,False,36.60,imagenet_subset_kaggle,1,resnet34_skips
8,20,finetuning,False,5.78,imagenet_subset_kaggle,1,resnet34_skips
10,20,replay,False,34.06,imagenet_subset_kaggle,1,resnet34_skips
12,20,ewc,False,7.18,imagenet_subset_kaggle,1,resnet34_skips


In [54]:
merged = pd.concat([df_noreg, filtered_df], axis=0)

In [55]:
pivot_table = pd.pivot_table(
    merged[merged["network"].str.contains("resnet")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    # aggfunc= lambda x: len(x),
    aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    fill_value=0,
)
pivot_table

dataset                   cifar100_fixed                          \
num_tasks                             5           10          20   
approach   is_regularized                                          
ewc        False              23.10±0.51  12.22±0.11   4.91±2.37   
finetuning False              20.55±0.35  11.54±0.10   5.03±0.61   
lwf        False              39.26±0.24  25.45±0.97  16.89±0.73   
replay     False              40.63±0.89  35.42±2.32  31.60±1.55   

dataset                   imagenet_subset_kaggle              
num_tasks                                     10          20  
approach   is_regularized                                     
ewc        False                      13.73±0.16   7.01±0.12  
finetuning False                      11.52±0.12   6.08±0.24  
lwf        False                      35.47±0.99  19.61±0.38  
replay     False                      38.59±0.72  35.00±1.76

In [58]:
pivot_table = pd.pivot_table(
    merged[merged["network"].str.contains("convnext")],
    index=["approach", "is_regularized"],
    columns=["dataset", "num_tasks"],
    values="avg_acc_tag",
    # aggfunc= lambda x: len(x),
    aggfunc=lambda x: f"{np.mean(x):.2f}±{np.std(x):.2f}",
    fill_value=0,
)
pivot_table

dataset                   cifar100_fixed                          \
num_tasks                             5           10          20   
approach   is_regularized                                          
ewc        False              29.57±0.03  16.92±0.55   8.87±0.53   
           True               31.12±0.16  20.87±0.41  14.36±0.20   
finetuning False              26.21±0.69  15.31±0.37   8.19±0.27   
           True               27.81±0.35  18.18±0.17  12.12±0.19   
lwf        False              40.70±0.09  29.83±0.21  23.01±0.87   
           True               40.88±0.46  30.11±0.45  21.63±0.48   
replay     False              34.53±0.25  28.59±0.13  25.42±0.25   
           True               33.02±0.33  26.28±0.37  24.64±0.25   

dataset                   imagenet_subset_kaggle              
num_tasks                                     10          20  
approach   is_regularized                                     
ewc        False                      21.65±0.70  10.81±0.39  
           True                       33.27±0.16  22.60±0.39  
finetuning False                      17.29±0.30   7.37±0.01  
           True                       26.97±0.82  15.71±0.85  
lwf        False                      43.73±0.46  29.60±0.71  
           True                       43.95±0.89  29.01±0.28  
replay     False                      34.55±0.86  29.78±0.74  
           True                       31.45±1.26  27.25±0.37

In [92]:
# merged.set_index("is_regularized").groupby(
#     ["num_tasks", "approach", "network", "dataset", "seed"]
# ).apply(lambda g: list(g["avg_acc_tag"]))
merged.set_index("is_regularized").groupby(
    ["num_tasks", "approach", "network", "dataset", "seed"]
).apply(
    lambda g: (
        g.loc[True, "avg_acc_tag"] - g.loc[False, "avg_acc_tag"]
        if True in g.index and False in g.index
        else None
    )
)

/var/folders/7p/77w4_2b565n0b8phw5c6dvtm0000gp/T/ipykernel_10652/3893530143.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged.set_index("is_regularized").groupby(["num_tasks","approach","network","dataset","seed"]).apply(lambda g: list(g["avg_acc_tag"]))


num_tasks  approach  network         dataset                 seed
5          ewc       convnext_tiny   cifar100_fixed          0                    [29.549999999999997, 31.28]
                                                             1       [29.549999999999997, 30.910000000000004]
                                                             2       [29.609999999999996, 31.180000000000003]
                     resnet34_skips  cifar100_fixed          0                                        [23.69]
                                                             1                                        [22.44]
                                                                                       ...                   
20         replay    resnet34_skips  cifar100_fixed          1                                        [29.65]
                                                             2                                        [31.72]
                                     imagenet_subset_k

In [87]:
merged.set_index("seed").groupby(["num_tasks", "approach", "network", "dataset"]).apply(
    lambda g: list(g["avg_acc_tag"])
)

/var/folders/7p/77w4_2b565n0b8phw5c6dvtm0000gp/T/ipykernel_10652/3098750605.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged.set_index("seed").groupby(["num_tasks","approach","network","dataset"]).apply(lambda g: list(g["avg_acc_tag"]))


num_tasks  approach    network         dataset               
5          ewc         convnext_tiny   cifar100_fixed            [29.549999999999997, 29.549999999999997, 29.60...
                       resnet34_skips  cifar100_fixed                           [22.44, 23.69, 23.159999999999997]
           finetuning  convnext_tiny   cifar100_fixed            [25.72, 25.72, 27.190000000000005, 27.4, 27.77...
                       resnet34_skips  cifar100_fixed                                        [20.69, 20.88, 20.07]
           lwf         convnext_tiny   cifar100_fixed            [40.63, 40.63, 40.83, 40.62, 41.52, 40.4899999...
                       resnet34_skips  cifar100_fixed                            [39.4, 39.45, 38.919999999999995]
           replay      convnext_tiny   cifar100_fixed            [34.699999999999996, 34.699999999999996, 34.18...
                       resnet34_skips  cifar100_fixed                           [41.28, 41.24, 39.370000000000005]
10         ewc    

In [89]:
merged[merged["seed"] == 0].groupby("num_")

,num_tasks,approach,is_regularized,avg_acc_tag,dataset,seed,network
0,5,finetuning,False,25.72,cifar100_fixed,0,convnext_tiny
2,5,replay,False,34.70,cifar100_fixed,0,convnext_tiny
4,5,ewc,False,29.55,cifar100_fixed,0,convnext_tiny
6,5,lwf,False,40.63,cifar100_fixed,0,convnext_tiny
8,10,finetuning,False,15.57,cifar100_fixed,0,convnext_tiny
10,10,replay,False,28.50,cifar100_fixed,0,convnext_tiny
12,10,ewc,False,16.53,cifar100_fixed,0,convnext_tiny
14,10,lwf,False,29.98,cifar100_fixed,0,convnext_tiny
16,20,finetuning,False,8.00,cifar100_fixed,0,convnext_tiny
18,20,replay,False,25.24,cifar100_fixed,0,convnext_tiny
